In [8]:
# @title Install or Update GABDConnect Libraries

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Function to install required packages
def install_packages(b):
    """
    Function to install or update necessary Python packages.

    Parameters:
    - b: Button object. Click event handler.
    """
    clear_output(wait=True)
    !pip install --upgrade pip -q  # Upgrade pip silently
    !pip install  git+https://github.com/Oriolrt/GABDConnect ucimlrepo -q  # Install or upgrade GABDConnect silently
    !pip install html2text -q  # Install or upgrade html2text silently
    print("Libraries have been downloaded or updated.")

install_packages(True)



ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Libraries have been downloaded or updated.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [9]:
from GABDConnect.oracleConnection import oracleConnection as orcl
import getpass
import logging
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import f1_score

from sklearn.metrics import accuracy_score
from datetime import datetime
import socket
from itertools import product
from ucimlrepo import fetch_ucirepo

In [10]:
import os


# Inicialitzem el diccionari amb les dades de connexió SSH per fer el tunel
SSH_USER = "student"
SSH_TUNNEL = "dcccluster.uab.cat"

PORT = "8195"
PWD = "vj2wkFAmstop6"
hostname= "oracle-1.grup03.gabd"


# Dades per fer el tunel SSH
ssh_server = {'ssh': SSH_TUNNEL , 'user': SSH_USER,
                'pwd': PWD, 'port': PORT} if SSH_TUNNEL is not None else None

# Dades de connexió a Oracle
user = '"gestorUCI"'  # @param {type:"string"}   Segons s'indica a l'enunciat de la pràctica
oracle_pwd = "gestor_password"  # @param {type:"string"}  Poseu aqui la contrasenya que hagueu assignat a l'usuari gestorUCI
serviceName="FREEPDB1" # @param {type:"string"}

In [11]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import json

def loadDataset(db,nameDataset: str ) -> tuple():

  #TODO:
  # Fes la consulta que toca i processa el resultat per retornar les matrius
  # numpy Xo, yo

  try:
    # Crear un cursor
    cursor = db.cursor()
    # Executar una consulta
    res = cursor.execute("SELECT * FROM dual")

    # TODO:
    # un cop hagueu comprovat que la consulta retorna el que ha de retornar comenteu
    # el següent codi
    for row in res:
      print(res)

    cursor.close()
  except Exception as e:
    print(e)
    dataset = fetch_ucirepo(name=nameDataset)
    Xo = dataset.data.features.to_numpy()
    labels = dataset.data.targets.to_numpy().reshape(-1)
    lut = {l: e for e, l in enumerate(np.unique(labels))}
    yo = np.array([lut[l] for l in labels])




  return Xo, yo


def saveExperiments(db,classificador : str = "Support Vector Machines" ,
                      iteracio: int= 0 ,
                      parametres_classificador: dict() = {'kernel': "linear",  'gamma': 1},
                      data_experiment: str ="",
                      f1_score : float = 0,
                      accuracy : float = 0) -> bool:

  # TODO:
  # Heu d'implementar la funció PL/SQL insertExperiment (o com la vulgueu anomenar)
  # Aquesta funció està en l'esquema de l'usuari GestorUCI i ha de ser visible i accessible
  # des de l'usuari testUCI que ha d'executar aquest script.
  try:
    # Crear un cursor
    cursor = db.cursor()

    # Cridar la funció PL/SQL
    resultat = cursor.callfunc("insertExperiment", bool, [classificador, iteracio,
                                                          parametres_classificador,
                                                          data_experiment,
                                                          f1_score, accuracy])

    # Tancar el cursor i la connexió
    cursor.close()
    return resultat
  except Exception as e:
    print(e)

  return False


In [12]:
# Cridem el constructor i obrim la connexió
db = orcl(user=user, passwd=oracle_pwd, hostname=hostname, ssh=ssh_server,serviceName=serviceName)

try:
  if not db.isStarted:
    db.open()
except Exception as e:
  print(e)


DPY-6005: cannot connect to database (CONNECTION_ID=EpGzJ2WRH+R0+85XTQFF+w==).
DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011


In [13]:
from collections import namedtuple
numIterations = 2 #@param {type:"number"}
Algorithms = {
    'Support Vector Machines': SVC,
    'K nearest Neighbor': KNN,
    'Random Forest': RFC
}
params = { 'SVC' : {'kernel': ("linear", "rbf", "poly"),
                    'gamma': [1, 5, 10, 20]},
            'KNeighborsClassifier' : { 'n_neighbors' : [3, 5, 10, 15]},
            'RandomForestClassifier' : {'max_depth': [2, 4, 10, None],
                    'criterion': ['gini', 'entropy', 'log_loss']}
            }

DATASETS = {"Iris":"Iris", "BreastCancer":"Breast Cancer Wisconsin (Diagnostic)",
            "Ionosphere":"Ionosphere", "Letter":"Letter Recognition"}


name = "Iris" #@param ["Iris", "BreastCancer", "Ionosphere", "Letter"] {type:"string"}
nameDataset = DATASETS[name]

Xo,yo=loadDataset(db,nameDataset)


n_sample = len(Xo)

np.random.seed(0)

current_time = datetime.now().strftime("%d/%m/%Y, %H:%M")

for k in Algorithms:
    classificador = Algorithms[k].__name__
    c_params = params[classificador].keys()
    for idx,values in enumerate(product(*params[classificador].values())):
        cp = {k: v for k, v in zip(c_params, values) if v is not None}
        clf = Algorithms[k](**cp)
        for i in range(numIterations):
            order = np.random.permutation(n_sample)
            X = Xo[order]
            y = yo[order].astype(float)

            X_train = X[: int(0.9 * n_sample)]
            y_train = y[: int(0.9 * n_sample)]
            X_test = X[int(0.9 * n_sample) :]
            y_test = y[int(0.9 * n_sample) :]


            # Si no executem el script a la màquina main de la pràctica visualitzem els resultats

            # fit the model
            clf.fit(X_train, y_train)

            y_pred = clf.predict(X_test)

            f1_s = f1_score(y_test, y_pred, average='macro')
            acc = accuracy_score(y_test, y_pred)

            fet = saveExperiments(db,classificador=k, iteracio=i,
                                  parametres_classificador=cp,
                                  data_experiment = current_time,
                                  f1_score=f1_s,
                                  accuracy = acc)
            if not fet:
              print(
                "Classificador: {}, Iteracio: {}, paràmetres: {}, time: {}, f-score: {}, accuracy: {}".format(k, i, cp, current_time, f1_s,
                                                                                            acc))

'NoneType' object has no attribute 'callproc'
'NoneType' object has no attribute 'callproc'
Classificador: Support Vector Machines, Iteracio: 0, paràmetres: {'kernel': 'linear', 'gamma': 1}, time: 03/12/2024, 10:48, f-score: 0.9326599326599326, accuracy: 0.9333333333333333
'NoneType' object has no attribute 'callproc'
Classificador: Support Vector Machines, Iteracio: 1, paràmetres: {'kernel': 'linear', 'gamma': 1}, time: 03/12/2024, 10:48, f-score: 0.9373219373219372, accuracy: 0.9333333333333333
'NoneType' object has no attribute 'callproc'
Classificador: Support Vector Machines, Iteracio: 0, paràmetres: {'kernel': 'linear', 'gamma': 5}, time: 03/12/2024, 10:48, f-score: 1.0, accuracy: 1.0
'NoneType' object has no attribute 'callproc'
Classificador: Support Vector Machines, Iteracio: 1, paràmetres: {'kernel': 'linear', 'gamma': 5}, time: 03/12/2024, 10:48, f-score: 1.0, accuracy: 1.0
'NoneType' object has no attribute 'callproc'
Classificador: Support Vector Machines, Iteracio: 0, par